In [12]:
import sys, os, csv, glob, json, uuid, pickle, math
import nltk 
import gensim, logging
import numpy as np, scipy, pandas as pd
from operator import itemgetter
from IPython.display import HTML, display
import tabulate  
from gensim.models import Doc2Vec
import json
import time
import re
import utils
import vocabulary
import nltk.tokenize.regexp
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

corpusdir = '../../all-the-news-dump/inputs/contents/' # Directory of corpus.

#this is quick
newcorpus = PlaintextCorpusReader(corpusdir, '.*')

In [17]:
utils.canonicalize_words(['Hi my name is STACY666! 500 million'])

['hi my name is stacy666! 500 million']

In [2]:
#code adapted from https://www.kaggle.com/girianantharaman/dual-embeddings-space-model-demo

## Create folders

In [2]:
CONTENT_INDEX = 9 #column number of article content 
csv.field_size_limit(sys.maxsize)
#filepath to store data so it's not all in memory
#make sure this is not in the github repo
CONTENT_PATH = '../../all-the-news-dump/inputs/contents/'
TOKENS_PATH = '../../all-the-news-dump/inputs/tokens/'
MODEL_PATH = '../../all-the-news-dump/model/'
PROCESSED_TOKENS_PATH = '../../all-the-news-dump/inputs/processed_tokens/'

if not os.path.exists(CONTENT_PATH):
    os.makedirs(CONTENT_PATH)
    
if not os.path.exists(TOKENS_PATH):
    os.makedirs(TOKENS_PATH)
    
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
if not os.path.exists(PROCESSED_TOKENS_PATH):
    os.makedirs(PROCESSED_TOKENS_PATH)

In [30]:
#about 10 min for 40k and 100k
#trying 100k - whole vocab is over 300k??
start = time.time()
vocab = utils.build_vocab(newcorpus, V = 100000)
print((time.time()-start)/60)

Vocabulary: 100,000 types
9.17759349346161


In [4]:
#may need to downoad punkt from nltk.download()
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt
      Package punkt is already up-to-date!

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [27]:
#read in data, store it in content and tokens folder
#content stores the whole articles
#tokens stores the whole article, but with each sentence on a different line.
count = 0

start = time.time()
fnames = ['../../.kaggle/datasets/snapcrack/all-the-news/articles1.csv', '../../.kaggle/datasets/snapcrack/all-the-news/articles2.csv', '../../.kaggle/datasets/snapcrack/all-the-news/articles3.csv']
for fname in fnames:
    f = open(fname)
    reader = csv.reader(f)
    next(reader, None)  # skip the headers
    for line in reader:
        count = count + 1 
        content = line[CONTENT_INDEX]
        cname = CONTENT_PATH + str(count) + '.txt'
        tname = TOKENS_PATH + str(count) + '.tokens'
        cf = open(cname, 'w')
        cf.write(content)
        cf.close()
        tf = open(tname, 'w')
        for sentence in nltk.sent_tokenize(content):
            tf.write("%s\n" % sentence.lower())
        tf.close()
        
print(time.time()-start)

282.6653096675873


In [62]:
#processed tokens
count = 0
tokenizer = nltk.tokenize.regexp.RegexpTokenizer(r'\w+')

nSentences = 0
nEmpties = 0

start = time.time()
fnames = ['../../.kaggle/datasets/snapcrack/all-the-news/articles1.csv', '../../.kaggle/datasets/snapcrack/all-the-news/articles2.csv', '../../.kaggle/datasets/snapcrack/all-the-news/articles3.csv']
for fname in fnames:
    print(fname)
    f = open(fname)
    reader = csv.reader(f)
    next(reader, None)  # skip the headers
    for line in reader:
        count = count + 1 
        content = line[CONTENT_INDEX]
        tname = PROCESSED_TOKENS_PATH + str(count) + '.ptokens'
        tf = open(tname, 'w')
        for sentence in nltk.sent_tokenize(content):
            if re.search('[a-zA-Z]', sentence) is not None:          
                sentence = tokenizer.tokenize(sentence)
                psentence = ' '.join(vocab.ids_to_words(utils.preprocess_sentences([sentence], vocab)))
                nSentences += 1
            else:
                psentence = ''
                nEmpties += 1
            tf.write("%s\n" % psentence)
        tf.close()
        
print((time.time()-start)/60)
print('# sentences: ', nSentences)
print('# empties: ', nEmpties)

../../.kaggle/datasets/snapcrack/all-the-news/articles1.csv
../../.kaggle/datasets/snapcrack/all-the-news/articles2.csv
../../.kaggle/datasets/snapcrack/all-the-news/articles3.csv
12.065627030531566
# sentences:  5000970
# empties:  69173


In [4]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in glob.iglob(self.dirname +'*.tokens', recursive=True):
            for line in open(fname):
                yield nltk.word_tokenize(line)

In [10]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = MySentences('../../all-the-news-dump/inputs/tokens/') 
model1 = gensim.models.Word2Vec(sentences, min_count=1)

2018-03-20 22:42:37,230 : INFO : collecting all words and their counts
2018-03-20 22:42:37,501 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-03-20 22:42:40,435 : INFO : PROGRESS: at sentence #10000, processed 242128 words, keeping 17565 word types
2018-03-20 22:42:43,685 : INFO : PROGRESS: at sentence #20000, processed 482320 words, keeping 25621 word types
2018-03-20 22:42:46,604 : INFO : PROGRESS: at sentence #30000, processed 724974 words, keeping 31215 word types
2018-03-20 22:42:49,460 : INFO : PROGRESS: at sentence #40000, processed 966583 words, keeping 36707 word types
2018-03-20 22:42:52,466 : INFO : PROGRESS: at sentence #50000, processed 1203495 words, keeping 40546 word types
2018-03-20 22:42:55,696 : INFO : PROGRESS: at sentence #60000, processed 1450809 words, keeping 43902 word types
2018-03-20 22:42:58,772 : INFO : PROGRESS: at sentence #70000, processed 1692830 words, keeping 47264 word types
2018-03-20 22:43:02,001 : INFO : PROGRESS: 

2018-03-20 22:46:15,177 : INFO : PROGRESS: at sentence #710000, processed 17455002 words, keeping 133881 word types
2018-03-20 22:46:18,199 : INFO : PROGRESS: at sentence #720000, processed 17696566 words, keeping 134831 word types
2018-03-20 22:46:21,276 : INFO : PROGRESS: at sentence #730000, processed 17939623 words, keeping 135683 word types
2018-03-20 22:46:24,553 : INFO : PROGRESS: at sentence #740000, processed 18184649 words, keeping 136389 word types
2018-03-20 22:46:27,563 : INFO : PROGRESS: at sentence #750000, processed 18427579 words, keeping 137445 word types
2018-03-20 22:46:30,675 : INFO : PROGRESS: at sentence #760000, processed 18667989 words, keeping 138272 word types
2018-03-20 22:46:33,669 : INFO : PROGRESS: at sentence #770000, processed 18907984 words, keeping 139108 word types
2018-03-20 22:46:37,024 : INFO : PROGRESS: at sentence #780000, processed 19148843 words, keeping 139862 word types
2018-03-20 22:46:40,499 : INFO : PROGRESS: at sentence #790000, processe

2018-03-20 22:49:53,947 : INFO : PROGRESS: at sentence #1420000, processed 34884072 words, keeping 184522 word types
2018-03-20 22:49:57,134 : INFO : PROGRESS: at sentence #1430000, processed 35128611 words, keeping 185063 word types
2018-03-20 22:50:00,374 : INFO : PROGRESS: at sentence #1440000, processed 35382579 words, keeping 185643 word types
2018-03-20 22:50:03,706 : INFO : PROGRESS: at sentence #1450000, processed 35629361 words, keeping 186247 word types
2018-03-20 22:50:06,748 : INFO : PROGRESS: at sentence #1460000, processed 35875321 words, keeping 186883 word types
2018-03-20 22:50:09,717 : INFO : PROGRESS: at sentence #1470000, processed 36117085 words, keeping 187611 word types
2018-03-20 22:50:12,739 : INFO : PROGRESS: at sentence #1480000, processed 36366475 words, keeping 188279 word types
2018-03-20 22:50:15,693 : INFO : PROGRESS: at sentence #1490000, processed 36615245 words, keeping 188924 word types
2018-03-20 22:50:18,668 : INFO : PROGRESS: at sentence #1500000,

2018-03-20 22:53:35,014 : INFO : PROGRESS: at sentence #2130000, processed 52338743 words, keeping 222740 word types
2018-03-20 22:53:37,909 : INFO : PROGRESS: at sentence #2140000, processed 52585413 words, keeping 223624 word types
2018-03-20 22:53:40,881 : INFO : PROGRESS: at sentence #2150000, processed 52828062 words, keeping 224064 word types
2018-03-20 22:53:44,043 : INFO : PROGRESS: at sentence #2160000, processed 53075394 words, keeping 224546 word types
2018-03-20 22:53:47,329 : INFO : PROGRESS: at sentence #2170000, processed 53320091 words, keeping 224969 word types
2018-03-20 22:53:50,435 : INFO : PROGRESS: at sentence #2180000, processed 53561599 words, keeping 225519 word types
2018-03-20 22:53:53,291 : INFO : PROGRESS: at sentence #2190000, processed 53800052 words, keeping 225946 word types
2018-03-20 22:53:56,194 : INFO : PROGRESS: at sentence #2200000, processed 54039019 words, keeping 226344 word types
2018-03-20 22:53:59,206 : INFO : PROGRESS: at sentence #2210000,

2018-03-20 22:57:19,546 : INFO : PROGRESS: at sentence #2840000, processed 69774564 words, keeping 255239 word types
2018-03-20 22:57:22,645 : INFO : PROGRESS: at sentence #2850000, processed 70027725 words, keeping 255609 word types
2018-03-20 22:57:25,660 : INFO : PROGRESS: at sentence #2860000, processed 70270138 words, keeping 256037 word types
2018-03-20 22:57:28,852 : INFO : PROGRESS: at sentence #2870000, processed 70517690 words, keeping 256492 word types
2018-03-20 22:57:31,941 : INFO : PROGRESS: at sentence #2880000, processed 70761077 words, keeping 256845 word types
2018-03-20 22:57:35,290 : INFO : PROGRESS: at sentence #2890000, processed 71004973 words, keeping 257319 word types
2018-03-20 22:57:38,365 : INFO : PROGRESS: at sentence #2900000, processed 71246699 words, keeping 257754 word types
2018-03-20 22:57:41,328 : INFO : PROGRESS: at sentence #2910000, processed 71486957 words, keeping 258179 word types
2018-03-20 22:57:44,226 : INFO : PROGRESS: at sentence #2920000,

...

2018-03-21 01:09:30,223 : INFO : EPOCH 5 - PROGRESS: at 74.89% examples, 66351 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:09:31,284 : INFO : EPOCH 5 - PROGRESS: at 74.97% examples, 66354 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:09:32,359 : INFO : EPOCH 5 - PROGRESS: at 75.05% examples, 66356 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:09:33,437 : INFO : EPOCH 5 - PROGRESS: at 75.13% examples, 66357 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:09:34,495 : INFO : EPOCH 5 - PROGRESS: at 75.21% examples, 66359 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:09:35,570 : INFO : EPOCH 5 - PROGRESS: at 75.29% examples, 66361 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:09:36,633 : INFO : EPOCH 5 - PROGRESS: at 75.37% examples, 66363 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:09:37,700 : INFO : EPOCH 5 - PROGRESS: at 75.45% examples, 66365 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:09:38,777 : INFO : EPOCH 5 - PROGRESS: at 75.53% examples, 66366 words/s, in_qsize 0, out_

2018-03-21 01:10:49,328 : INFO : EPOCH 5 - PROGRESS: at 80.69% examples, 66385 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:10:50,405 : INFO : EPOCH 5 - PROGRESS: at 80.77% examples, 66386 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:10:51,450 : INFO : EPOCH 5 - PROGRESS: at 80.85% examples, 66390 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:10:52,506 : INFO : EPOCH 5 - PROGRESS: at 80.93% examples, 66392 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:10:53,558 : INFO : EPOCH 5 - PROGRESS: at 81.01% examples, 66395 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:10:54,614 : INFO : EPOCH 5 - PROGRESS: at 81.09% examples, 66397 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:10:55,631 : INFO : EPOCH 5 - PROGRESS: at 81.16% examples, 66401 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:10:56,684 : INFO : EPOCH 5 - PROGRESS: at 81.24% examples, 66404 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:10:57,752 : INFO : EPOCH 5 - PROGRESS: at 81.32% examples, 66406 words/s, in_qsize 0, out_

2018-03-21 01:12:08,514 : INFO : EPOCH 5 - PROGRESS: at 86.51% examples, 66433 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:12:09,593 : INFO : EPOCH 5 - PROGRESS: at 86.59% examples, 66434 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:12:10,670 : INFO : EPOCH 5 - PROGRESS: at 86.67% examples, 66435 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:12:11,745 : INFO : EPOCH 5 - PROGRESS: at 86.75% examples, 66436 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:12:12,830 : INFO : EPOCH 5 - PROGRESS: at 86.84% examples, 66437 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:12:13,909 : INFO : EPOCH 5 - PROGRESS: at 86.92% examples, 66438 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:12:14,980 : INFO : EPOCH 5 - PROGRESS: at 86.99% examples, 66439 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:12:16,049 : INFO : EPOCH 5 - PROGRESS: at 87.07% examples, 66441 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:12:17,113 : INFO : EPOCH 5 - PROGRESS: at 87.16% examples, 66443 words/s, in_qsize 0, out_

2018-03-21 01:13:26,597 : INFO : EPOCH 5 - PROGRESS: at 92.13% examples, 66385 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:13:27,666 : INFO : EPOCH 5 - PROGRESS: at 92.21% examples, 66386 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:13:28,747 : INFO : EPOCH 5 - PROGRESS: at 92.28% examples, 66387 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:13:29,849 : INFO : EPOCH 5 - PROGRESS: at 92.36% examples, 66386 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:13:30,915 : INFO : EPOCH 5 - PROGRESS: at 92.44% examples, 66388 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:13:31,977 : INFO : EPOCH 5 - PROGRESS: at 92.53% examples, 66390 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:13:33,039 : INFO : EPOCH 5 - PROGRESS: at 92.61% examples, 66392 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:13:34,105 : INFO : EPOCH 5 - PROGRESS: at 92.69% examples, 66393 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:13:35,193 : INFO : EPOCH 5 - PROGRESS: at 92.77% examples, 66394 words/s, in_qsize 0, out_

2018-03-21 01:14:45,564 : INFO : EPOCH 5 - PROGRESS: at 97.83% examples, 66319 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:14:46,639 : INFO : EPOCH 5 - PROGRESS: at 97.91% examples, 66320 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:14:47,713 : INFO : EPOCH 5 - PROGRESS: at 97.99% examples, 66321 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:14:48,808 : INFO : EPOCH 5 - PROGRESS: at 98.07% examples, 66322 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:14:49,834 : INFO : EPOCH 5 - PROGRESS: at 98.14% examples, 66320 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:14:50,958 : INFO : EPOCH 5 - PROGRESS: at 98.21% examples, 66313 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:14:52,040 : INFO : EPOCH 5 - PROGRESS: at 98.27% examples, 66298 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:14:53,047 : INFO : EPOCH 5 - PROGRESS: at 98.33% examples, 66287 words/s, in_qsize 0, out_qsize 0
2018-03-21 01:14:54,089 : INFO : EPOCH 5 - PROGRESS: at 98.39% examples, 66273 words/s, in_qsize 0, out_

In [15]:
model1.save('../../all-the-news-dump/w2v-lc.model')
model1.wv.save_word2vec_format('../../all-the-news-dump/w2v-lc.model.bin', binary=True)
vocab = dict([(k, v.index) for k, v in model1.wv.vocab.items()])
with open('../../all-the-news-dump/w2v-lc-vocab.json', 'w') as f:
    f.write(json.dumps(vocab))

2018-03-21 01:31:41,424 : INFO : saving Word2Vec object under ../../all-the-news-dump/w2v-lc.model, separately None
2018-03-21 01:31:41,426 : INFO : storing np array 'vectors' to ../../all-the-news-dump/w2v-lc.model.wv.vectors.npy
2018-03-21 01:31:41,681 : INFO : not storing attribute vectors_norm
2018-03-21 01:31:41,685 : INFO : storing np array 'syn1neg' to ../../all-the-news-dump/w2v-lc.model.trainables.syn1neg.npy
2018-03-21 01:31:43,314 : INFO : saved ../../all-the-news-dump/w2v-lc.model
2018-03-21 01:31:43,318 : INFO : storing 340064x100 projection weights into ../../all-the-news-dump/w2v-lc.model.bin


In [5]:
model1 = gensim.models.Word2Vec.load('../../all-the-news-dump/w2v-lc.model')

In [20]:
model1.wv.most_similar(positive=['obama', 'democrat'], negative=['clinton'])

[('republican', 0.5808690786361694),
 ('senator', 0.5652804970741272),
 ('calheiros', 0.555640697479248),
 ('incumbent', 0.5480653047561646),
 ('wagle', 0.5384800434112549),
 ('congressman', 0.536839485168457),
 ('brownback', 0.5267091393470764),
 ('outgoing', 0.5252715945243835),
 ('segregationist', 0.514189600944519),
 ('lawmaker', 0.5140193700790405)]

In [60]:
model1.wv.most_similar('conservative')

[('liberal', 0.8809554576873779),
 ('progressive', 0.8421754837036133),
 ('rightwing', 0.7585837841033936),
 ('libertarian', 0.7405217885971069),
 ('centrist', 0.7322021722793579),
 ('leftwing', 0.6988232135772705),
 ('leftist', 0.6965807676315308),
 ('fringe', 0.6776911020278931),
 ('neoconservative', 0.675796389579773),
 ('evangelical', 0.6756139993667603)]

In [60]:
re.search('[a-zA-Z]', ' ?.') is not None

False

In [11]:
count = 1
folder = '../../all-the-news-dump/inputs/processed_tokens/'
for fname in glob.iglob(folder +'*.ptokens', recursive=True):
            if count > 2:
                break
            count += 1
            for uid, line in enumerate(open(fname)):
               
                content = fname[len(folder):(len(fname)-8)]
                tag = 'CONTENT_' + str(content) + '_TOKEN_' + str(uid)
                print(gensim.models.doc2vec.TaggedDocument(words=line.split(), tags=[tag]))

TaggedDocument(['<s>', 'for', 'such', 'a', 'commonplace', 'bodily', 'function', 'the', 'sneeze', 'has', 'messed', 'with', 'our', 'minds', 'and', 'noses', 'for', 'centuries', '<s>'], ['CONTENT_112928_TOKEN_0'])
TaggedDocument(['<s>', 'it', 'will', 'kill', 'us', 'it', 'won', 't', 'kill', 'us', '<s>'], ['CONTENT_112928_TOKEN_1'])
TaggedDocument(['<s>', 'we', 'll', 'have', 'bad', 'luck', 'we', 'll', 'have', 'good', 'luck', '<s>'], ['CONTENT_112928_TOKEN_2'])
TaggedDocument(['<s>', 'watch', 'out', 'for', 'satan', 'he', 's', 'wily', 'and', 'knows', 'how', 'to', 'get', 'into', 'your', 'nasal', 'membranes', '<s>'], ['CONTENT_112928_TOKEN_3'])
TaggedDocument(['<s>', 'did', 'you', 'have', 'too', 'much', 'to', 'eat', '<s>'], ['CONTENT_112928_TOKEN_4'])
TaggedDocument(['<s>', 'are', 'you', 'sad', '<s>'], ['CONTENT_112928_TOKEN_5'])
TaggedDocument(['<s>', 'do', 'you', 'have', 'a', 'weak', 'heart', '<s>'], ['CONTENT_112928_TOKEN_6'])
TaggedDocument(['<s>', 'technically', 'sneezing', 'is', 'a', 'refl

In [6]:
class LabeledLineSentence_TEST(object):
    def __init__(self, dirname):
        self.dirname = dirname
    def __iter__(self):
        count = 1
        for fname in glob.iglob(self.dirname +'*.ptokens', recursive=True):
            #if count > 25000:
            #    break
            count += 1
            for uid, line in enumerate(open(fname)):
                if re.search('[a-zA-Z]', line) is not None:
                    content = fname[len(self.dirname):(len(fname)-8)]
                    tag = 'CONTENT_' + str(content) + '_TOKEN_' + str(uid)
                    yield gensim.models.doc2vec.TaggedDocument(words=line.split(), tags=[tag])

In [7]:
sentences = LabeledLineSentence_TEST('../../all-the-news-dump/inputs/processed_tokens/')

In [10]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
modeld2v = Doc2Vec(alpha=0.025, min_alpha=0.025)  # use fixed learning rate
start = time.time()
modeld2v.build_vocab(sentences)
print((time.time()-start)/60)

2018-04-04 19:07:06,932 : INFO : collecting all words and their counts
2018-04-04 19:07:07,203 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-04-04 19:07:07,367 : INFO : PROGRESS: at example #10000, processed 231069 words (1421807/s), 17019 word types, 10000 tags
2018-04-04 19:07:07,534 : INFO : PROGRESS: at example #20000, processed 459283 words (1375043/s), 23640 word types, 20000 tags
2018-04-04 19:07:07,705 : INFO : PROGRESS: at example #30000, processed 692271 words (1369101/s), 28192 word types, 30000 tags
2018-04-04 19:07:07,887 : INFO : PROGRESS: at example #40000, processed 928635 words (1308768/s), 31981 word types, 40000 tags
2018-04-04 19:07:08,069 : INFO : PROGRESS: at example #50000, processed 1164195 words (1301435/s), 35381 word types, 50000 tags
2018-04-04 19:07:08,237 : INFO : PROGRESS: at example #60000, processed 1414579 words (1496073/s), 38002 word types, 60000 tags
2018-04-04 19:07:08,402 : INFO : PROGRESS: at example #70000,

2018-04-04 19:07:17,815 : INFO : PROGRESS: at example #630000, processed 14673682 words (1338673/s), 79546 word types, 630000 tags
2018-04-04 19:07:17,989 : INFO : PROGRESS: at example #640000, processed 14909376 words (1358311/s), 79862 word types, 640000 tags
2018-04-04 19:07:18,156 : INFO : PROGRESS: at example #650000, processed 15144201 words (1414492/s), 80094 word types, 650000 tags
2018-04-04 19:07:18,340 : INFO : PROGRESS: at example #660000, processed 15380080 words (1295126/s), 80384 word types, 660000 tags
2018-04-04 19:07:18,512 : INFO : PROGRESS: at example #670000, processed 15608270 words (1332886/s), 80640 word types, 670000 tags
2018-04-04 19:07:18,686 : INFO : PROGRESS: at example #680000, processed 15835274 words (1311366/s), 80919 word types, 680000 tags
2018-04-04 19:07:18,866 : INFO : PROGRESS: at example #690000, processed 16073010 words (1324997/s), 81219 word types, 690000 tags
2018-04-04 19:07:19,095 : INFO : PROGRESS: at example #700000, processed 16302780 w

2018-04-04 19:07:29,685 : INFO : PROGRESS: at example #1260000, processed 29340218 words (1251528/s), 90026 word types, 1260000 tags
2018-04-04 19:07:29,865 : INFO : PROGRESS: at example #1270000, processed 29573491 words (1300864/s), 90127 word types, 1270000 tags
2018-04-04 19:07:30,045 : INFO : PROGRESS: at example #1280000, processed 29816587 words (1362396/s), 90199 word types, 1280000 tags
2018-04-04 19:07:30,231 : INFO : PROGRESS: at example #1290000, processed 30049406 words (1253225/s), 90283 word types, 1290000 tags
2018-04-04 19:07:30,413 : INFO : PROGRESS: at example #1300000, processed 30279741 words (1273402/s), 90346 word types, 1300000 tags
2018-04-04 19:07:30,599 : INFO : PROGRESS: at example #1310000, processed 30509579 words (1247668/s), 90430 word types, 1310000 tags
2018-04-04 19:07:30,783 : INFO : PROGRESS: at example #1320000, processed 30747992 words (1300648/s), 90553 word types, 1320000 tags
2018-04-04 19:07:31,254 : INFO : PROGRESS: at example #1330000, proce

2018-04-04 19:07:41,249 : INFO : PROGRESS: at example #1880000, processed 43800439 words (1396395/s), 94130 word types, 1880000 tags
2018-04-04 19:07:41,413 : INFO : PROGRESS: at example #1890000, processed 44039467 words (1466951/s), 94172 word types, 1890000 tags
2018-04-04 19:07:41,577 : INFO : PROGRESS: at example #1900000, processed 44273948 words (1439689/s), 94221 word types, 1900000 tags
2018-04-04 19:07:41,747 : INFO : PROGRESS: at example #1910000, processed 44507913 words (1385559/s), 94260 word types, 1910000 tags
2018-04-04 19:07:41,910 : INFO : PROGRESS: at example #1920000, processed 44737125 words (1409382/s), 94304 word types, 1920000 tags
2018-04-04 19:07:42,073 : INFO : PROGRESS: at example #1930000, processed 44964176 words (1406237/s), 94349 word types, 1930000 tags
2018-04-04 19:07:42,239 : INFO : PROGRESS: at example #1940000, processed 45191210 words (1370651/s), 94391 word types, 1940000 tags
2018-04-04 19:07:42,406 : INFO : PROGRESS: at example #1950000, proce

2018-04-04 19:07:52,593 : INFO : PROGRESS: at example #2500000, processed 58212008 words (1381041/s), 96326 word types, 2500000 tags
2018-04-04 19:07:52,765 : INFO : PROGRESS: at example #2510000, processed 58443032 words (1344199/s), 96355 word types, 2510000 tags
2018-04-04 19:07:52,940 : INFO : PROGRESS: at example #2520000, processed 58680162 words (1364119/s), 96385 word types, 2520000 tags
2018-04-04 19:07:53,130 : INFO : PROGRESS: at example #2530000, processed 58917853 words (1260562/s), 96419 word types, 2530000 tags
2018-04-04 19:07:53,322 : INFO : PROGRESS: at example #2540000, processed 59145946 words (1191572/s), 96437 word types, 2540000 tags
2018-04-04 19:07:53,511 : INFO : PROGRESS: at example #2550000, processed 59378242 words (1235453/s), 96459 word types, 2550000 tags
2018-04-04 19:07:53,688 : INFO : PROGRESS: at example #2560000, processed 59616918 words (1357872/s), 96490 word types, 2560000 tags
2018-04-04 19:07:53,870 : INFO : PROGRESS: at example #2570000, proce

2018-04-04 19:08:05,025 : INFO : PROGRESS: at example #3120000, processed 72611402 words (1312661/s), 97579 word types, 3120000 tags
2018-04-04 19:08:05,217 : INFO : PROGRESS: at example #3130000, processed 72844160 words (1217927/s), 97594 word types, 3130000 tags
2018-04-04 19:08:05,408 : INFO : PROGRESS: at example #3140000, processed 73074198 words (1207412/s), 97617 word types, 3140000 tags
2018-04-04 19:08:05,604 : INFO : PROGRESS: at example #3150000, processed 73310868 words (1213774/s), 97630 word types, 3150000 tags
2018-04-04 19:08:05,805 : INFO : PROGRESS: at example #3160000, processed 73542134 words (1163225/s), 97646 word types, 3160000 tags
2018-04-04 19:08:05,995 : INFO : PROGRESS: at example #3170000, processed 73773861 words (1223978/s), 97659 word types, 3170000 tags
2018-04-04 19:08:06,180 : INFO : PROGRESS: at example #3180000, processed 74005349 words (1260189/s), 97676 word types, 3180000 tags
2018-04-04 19:08:06,357 : INFO : PROGRESS: at example #3190000, proce

2018-04-04 19:08:16,578 : INFO : PROGRESS: at example #3740000, processed 87053019 words (1380141/s), 98398 word types, 3740000 tags
2018-04-04 19:08:16,746 : INFO : PROGRESS: at example #3750000, processed 87286409 words (1401311/s), 98406 word types, 3750000 tags
2018-04-04 19:08:16,910 : INFO : PROGRESS: at example #3760000, processed 87526873 words (1480467/s), 98416 word types, 3760000 tags
2018-04-04 19:08:17,071 : INFO : PROGRESS: at example #3770000, processed 87760021 words (1458150/s), 98424 word types, 3770000 tags
2018-04-04 19:08:17,244 : INFO : PROGRESS: at example #3780000, processed 88000636 words (1395811/s), 98432 word types, 3780000 tags
2018-04-04 19:08:17,414 : INFO : PROGRESS: at example #3790000, processed 88230094 words (1357748/s), 98438 word types, 3790000 tags
2018-04-04 19:08:17,588 : INFO : PROGRESS: at example #3800000, processed 88468674 words (1379119/s), 98445 word types, 3800000 tags
2018-04-04 19:08:17,749 : INFO : PROGRESS: at example #3810000, proce

2018-04-04 19:08:27,374 : INFO : PROGRESS: at example #4360000, processed 101553582 words (1323824/s), 98937 word types, 4360000 tags
2018-04-04 19:08:27,546 : INFO : PROGRESS: at example #4370000, processed 101787129 words (1370626/s), 98947 word types, 4370000 tags
2018-04-04 19:08:27,718 : INFO : PROGRESS: at example #4380000, processed 102021308 words (1369072/s), 98961 word types, 4380000 tags
2018-04-04 19:08:27,891 : INFO : PROGRESS: at example #4390000, processed 102253450 words (1342989/s), 98969 word types, 4390000 tags
2018-04-04 19:08:28,069 : INFO : PROGRESS: at example #4400000, processed 102486494 words (1322621/s), 98985 word types, 4400000 tags
2018-04-04 19:08:28,231 : INFO : PROGRESS: at example #4410000, processed 102700720 words (1327875/s), 98988 word types, 4410000 tags
2018-04-04 19:08:28,404 : INFO : PROGRESS: at example #4420000, processed 102932964 words (1348541/s), 98997 word types, 4420000 tags
2018-04-04 19:08:29,345 : INFO : PROGRESS: at example #4430000

2018-04-04 19:08:39,019 : INFO : PROGRESS: at example #4980000, processed 115997644 words (1336714/s), 99366 word types, 4980000 tags
2018-04-04 19:08:39,188 : INFO : PROGRESS: at example #4990000, processed 116224840 words (1349843/s), 99368 word types, 4990000 tags
2018-04-04 19:08:39,363 : INFO : PROGRESS: at example #5000000, processed 116453791 words (1316515/s), 99374 word types, 5000000 tags
2018-04-04 19:08:39,382 : INFO : collected 99374 word types and 5000970 unique tags from a corpus of 5000970 examples and 116475319 words
2018-04-04 19:08:39,383 : INFO : Loading a fresh vocabulary
2018-04-04 19:08:39,567 : INFO : min_count=5 retains 99368 unique words (99% of original 99374, drops 6)
2018-04-04 19:08:39,568 : INFO : min_count=5 leaves 116475301 word corpus (99% of original 116475319, drops 18)
2018-04-04 19:08:39,873 : INFO : deleting the raw counts dictionary of 99374 items
2018-04-04 19:08:39,877 : INFO : sample=0.001 downsamples 41 most-common words
2018-04-04 19:08:39,8

2.4361098249753317


In [11]:
start = time.time()
num_epochs = 10
for i in range(num_epochs):
    print(i+1)
    modeld2v.train(sentences, total_examples=modeld2v.corpus_count, epochs=1)
    modeld2v.alpha -= 0.002  # decrease the learning rate
    modeld2v.min_alpha = modeld2v.alpha  # fix the learning rate, no decay
modeld2v.save('../../all-the-news-dump/d2v-full_utils.model')
print((time.time()-start)/60)

2018-04-04 19:09:33,174 : INFO : training model with 3 workers on 99368 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5


1


2018-04-04 19:09:34,213 : INFO : EPOCH 1 - PROGRESS: at 0.14% examples, 119875 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:09:35,265 : INFO : EPOCH 1 - PROGRESS: at 0.37% examples, 159713 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:09:36,289 : INFO : EPOCH 1 - PROGRESS: at 0.59% examples, 169282 words/s, in_qsize 6, out_qsize 0
2018-04-04 19:09:37,293 : INFO : EPOCH 1 - PROGRESS: at 0.81% examples, 177076 words/s, in_qsize 6, out_qsize 0
2018-04-04 19:09:38,302 : INFO : EPOCH 1 - PROGRESS: at 1.02% examples, 180020 words/s, in_qsize 6, out_qsize 0
2018-04-04 19:09:39,315 : INFO : EPOCH 1 - PROGRESS: at 1.21% examples, 180592 words/s, in_qsize 6, out_qsize 0
2018-04-04 19:09:40,325 : INFO : EPOCH 1 - PROGRESS: at 1.40% examples, 178913 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:09:41,342 : INFO : EPOCH 1 - PROGRESS: at 1.59% examples, 178479 words/s, in_qsize 6, out_qsize 0
2018-04-04 19:09:42,378 : INFO : EPOCH 1 - PROGRESS: at 1.80% examples, 178617 words/s, in_qsize 5, out_

2018-04-04 19:10:49,793 : INFO : EPOCH 1 - PROGRESS: at 15.20% examples, 179460 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:10:50,808 : INFO : EPOCH 1 - PROGRESS: at 15.43% examples, 179802 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:10:51,831 : INFO : EPOCH 1 - PROGRESS: at 15.66% examples, 180128 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:10:52,839 : INFO : EPOCH 1 - PROGRESS: at 15.88% examples, 180369 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:10:53,848 : INFO : EPOCH 1 - PROGRESS: at 16.10% examples, 180611 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:10:54,888 : INFO : EPOCH 1 - PROGRESS: at 16.34% examples, 180873 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:10:55,890 : INFO : EPOCH 1 - PROGRESS: at 16.56% examples, 181119 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:10:56,904 : INFO : EPOCH 1 - PROGRESS: at 16.78% examples, 181332 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:10:57,954 : INFO : EPOCH 1 - PROGRESS: at 16.97% examples, 181193 words/s, in_qsiz

2018-04-04 19:12:05,650 : INFO : EPOCH 1 - PROGRESS: at 30.14% examples, 178928 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:12:06,687 : INFO : EPOCH 1 - PROGRESS: at 30.37% examples, 179085 words/s, in_qsize 6, out_qsize 0
2018-04-04 19:12:07,704 : INFO : EPOCH 1 - PROGRESS: at 30.60% examples, 179261 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:12:08,745 : INFO : EPOCH 1 - PROGRESS: at 30.83% examples, 179411 words/s, in_qsize 6, out_qsize 0
2018-04-04 19:12:09,764 : INFO : EPOCH 1 - PROGRESS: at 31.07% examples, 179578 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:12:10,779 : INFO : EPOCH 1 - PROGRESS: at 31.29% examples, 179750 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:12:11,802 : INFO : EPOCH 1 - PROGRESS: at 31.48% examples, 179666 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:12:12,832 : INFO : EPOCH 1 - PROGRESS: at 31.68% examples, 179629 words/s, in_qsize 5, out_qsize 0
2018-04-04 19:12:13,834 : INFO : EPOCH 1 - PROGRESS: at 31.89% examples, 179714 words/s, in_qsiz

...

2018-04-04 20:35:39,254 : INFO : EPOCH 1 - PROGRESS: at 54.08% examples, 160818 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:35:40,281 : INFO : EPOCH 1 - PROGRESS: at 54.28% examples, 160862 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:35:41,368 : INFO : EPOCH 1 - PROGRESS: at 54.45% examples, 160824 words/s, in_qsize 6, out_qsize 0
2018-04-04 20:35:42,456 : INFO : EPOCH 1 - PROGRESS: at 54.65% examples, 160833 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:35:43,479 : INFO : EPOCH 1 - PROGRESS: at 54.84% examples, 160854 words/s, in_qsize 6, out_qsize 0
2018-04-04 20:35:44,545 : INFO : EPOCH 1 - PROGRESS: at 55.03% examples, 160877 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:35:45,569 : INFO : EPOCH 1 - PROGRESS: at 55.22% examples, 160895 words/s, in_qsize 6, out_qsize 0
2018-04-04 20:35:46,570 : INFO : EPOCH 1 - PROGRESS: at 55.39% examples, 160876 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:35:47,571 : INFO : EPOCH 1 - PROGRESS: at 55.56% examples, 160857 words/s, in_qsiz

2018-04-04 20:36:55,306 : INFO : EPOCH 1 - PROGRESS: at 67.47% examples, 160411 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:36:56,344 : INFO : EPOCH 1 - PROGRESS: at 67.66% examples, 160441 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:36:57,366 : INFO : EPOCH 1 - PROGRESS: at 67.85% examples, 160437 words/s, in_qsize 6, out_qsize 0
2018-04-04 20:36:58,405 : INFO : EPOCH 1 - PROGRESS: at 68.03% examples, 160447 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:36:59,407 : INFO : EPOCH 1 - PROGRESS: at 68.22% examples, 160474 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:37:00,418 : INFO : EPOCH 1 - PROGRESS: at 68.41% examples, 160496 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:37:01,420 : INFO : EPOCH 1 - PROGRESS: at 68.61% examples, 160540 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:37:02,459 : INFO : EPOCH 1 - PROGRESS: at 68.78% examples, 160510 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:37:03,498 : INFO : EPOCH 1 - PROGRESS: at 68.96% examples, 160499 words/s, in_qsiz

2018-04-04 20:38:12,429 : INFO : EPOCH 1 - PROGRESS: at 80.96% examples, 160091 words/s, in_qsize 6, out_qsize 0
2018-04-04 20:38:13,475 : INFO : EPOCH 1 - PROGRESS: at 81.15% examples, 160097 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:38:14,519 : INFO : EPOCH 1 - PROGRESS: at 81.35% examples, 160138 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:38:15,529 : INFO : EPOCH 1 - PROGRESS: at 81.53% examples, 160141 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:38:16,571 : INFO : EPOCH 1 - PROGRESS: at 81.72% examples, 160151 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:38:17,639 : INFO : EPOCH 1 - PROGRESS: at 81.90% examples, 160133 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:38:18,697 : INFO : EPOCH 1 - PROGRESS: at 82.08% examples, 160119 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:38:19,745 : INFO : EPOCH 1 - PROGRESS: at 82.26% examples, 160125 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:38:20,796 : INFO : EPOCH 1 - PROGRESS: at 82.44% examples, 160113 words/s, in_qsiz

2018-04-04 20:39:28,224 : INFO : EPOCH 1 - PROGRESS: at 94.67% examples, 160628 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:39:29,237 : INFO : EPOCH 1 - PROGRESS: at 94.85% examples, 160642 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:39:30,252 : INFO : EPOCH 1 - PROGRESS: at 95.02% examples, 160627 words/s, in_qsize 6, out_qsize 0
2018-04-04 20:39:31,282 : INFO : EPOCH 1 - PROGRESS: at 95.22% examples, 160650 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:39:32,285 : INFO : EPOCH 1 - PROGRESS: at 95.40% examples, 160654 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:39:33,308 : INFO : EPOCH 1 - PROGRESS: at 95.59% examples, 160680 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:39:34,367 : INFO : EPOCH 1 - PROGRESS: at 95.79% examples, 160695 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:39:35,394 : INFO : EPOCH 1 - PROGRESS: at 95.98% examples, 160720 words/s, in_qsize 5, out_qsize 0
2018-04-04 20:39:36,443 : INFO : EPOCH 1 - PROGRESS: at 96.17% examples, 160723 words/s, in_qsiz

91.16190029382706


In [36]:
print((time.time()-start)/60)
#for 5 epochs

155.86895682414374


In [37]:
modeld2v.save('../../all-the-news-dump/d2v-full-5epochs.model')
#model1.wv.save_word2vec_format('../../all-the-news-dump/w2v-lc.model.bin', binary=True)
#vocab = dict([(k, v.index) for k, v in model1.wv.vocab.items()])
#with open('../../all-the-news-dump/w2v-lc-vocab.json', 'w') as f:
#    f.write(json.dumps(vocab))

2018-03-31 00:05:45,117 : INFO : saving Doc2Vec object under ../../all-the-news-dump/d2v-full-5epochs.model, separately None
2018-03-31 00:05:45,120 : INFO : storing np array 'syn1neg' to ../../all-the-news-dump/d2v-full-5epochs.model.trainables.syn1neg.npy
2018-03-31 00:05:45,175 : INFO : storing np array 'vectors' to ../../all-the-news-dump/d2v-full-5epochs.model.wv.vectors.npy
2018-03-31 00:05:45,228 : INFO : storing np array 'vectors_docs' to ../../all-the-news-dump/d2v-full-5epochs.model.docvecs.vectors_docs.npy
2018-03-31 00:06:08,500 : INFO : saved ../../all-the-news-dump/d2v-full-5epochs.model


In [38]:
modeld2v.docvecs['CONTENT_55355_TOKEN_0']

array([ -2.63779052e-03,   7.63207849e-04,   2.31453986e-03,
         1.35988672e-03,  -1.06009736e-03,  -4.28857841e-03,
         1.57091988e-03,  -4.29542642e-03,  -2.66154110e-03,
        -3.89073743e-03,  -1.57311704e-04,  -4.35791258e-03,
        -4.90500452e-03,   4.86594019e-03,   2.80455034e-03,
        -1.90040201e-03,  -3.94264283e-03,   2.21770373e-03,
        -3.83980363e-03,  -2.12597474e-03,   4.82200459e-03,
        -7.35231326e-04,  -2.04554317e-03,  -4.14188020e-03,
        -3.51063418e-03,  -2.80093378e-03,  -2.96322489e-03,
        -1.74974848e-03,  -1.70252461e-03,   3.63943377e-03,
        -1.94267859e-03,   2.05455697e-03,  -3.51610989e-03,
        -8.31603480e-04,  -2.17608031e-04,  -2.78658653e-03,
        -7.38951727e-04,   2.85908987e-04,   3.20582883e-03,
         9.16072167e-04,   3.48675414e-03,  -1.89297099e-03,
        -2.26070429e-03,   4.57902625e-03,   1.75741932e-03,
        -1.59767907e-04,   3.33786273e-04,   3.08208470e-03,
         2.50869780e-03,